# 1) IMPORTING LIBRARIES AND PACKAGES

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pickle
from sklearn import model_selection
import matplotlib.pyplot as plt
import model_utils
from geopy.geocoders import Nominatim, GeoNames
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import great_circle
import lightgbm as lgb

# 2) DATASET IMPORTING

In [3]:
data = pd.read_csv('Data/DataCoSupplyChainDataset.csv',header= 0,encoding='unicode_escape')
pd.set_option('display.max_columns',None)
data.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.110000,0.04,180517,327.75,0.29,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.940001,0.07,179252,327.75,0.08,1,327.75,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.500000,0.09,179251,327.75,0.45,1,327.75,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


# 3) FEATURE ENGINEERING

Convert dates

In [4]:
#Converting categorical features that represent date and time to datetime datatype.
data['order_date'] = pd.to_datetime(data['order date (DateOrders)'])
# Handling Time and date variables
data['order_year'] = pd.DatetimeIndex(data['order_date']).year
data['order_month'] = pd.DatetimeIndex(data['order_date']).month
data['order_day'] = pd.DatetimeIndex(data['order_date']).day

Is it imbalanced?

In [5]:
# Check percentage late_delivery_risk vs non_late_delivery_risk
late_delivery_risk = data['Late_delivery_risk'].value_counts()
print(late_delivery_risk)
print('Percentage of Late Delivery Risk :',round(late_delivery_risk[1]/data.shape[0]*100,2))
perc_late_delivery_risk = round(late_delivery_risk[1]/data.shape[0]*100,2)

Late_delivery_risk
1    98977
0    81542
Name: count, dtype: int64
Percentage of Late Delivery Risk : 54.83


Calculate distances

In [6]:
# Open city coordinates from a file
with open('Data/city_coordinates.pkl', 'rb') as f:
    city_coordinates = pickle.load(f)
city_coordinates = pd.DataFrame(city_coordinates.items(), columns=['Order City', 'Coordinates'])

country_coordinates = pd.read_csv('Data/country_coordinates.csv')

In [7]:
data = pd.merge(data, city_coordinates, on='Order City', how='left')
# Fill in missing city coordinates
data['Coordinates'] = data['Coordinates'].fillna(data['Order Country'].map(country_coordinates.set_index('Order Country')['Coordinates']))
# Fill in values that are not two numbers (latitude, longitude) with (0, 0) (also replace (None, None) with (0, 0))
data['Coordinates'] = data['Coordinates'].apply(lambda x: (0, 0) if not isinstance(x, tuple) or len(x) != 2 else x)
# Fill in values that are contain None (for example (None, None)) with (0, 0)
data['Coordinates'] = data['Coordinates'].apply(lambda x: (0, 0) if x[0] is None else x)
# Split the coordinates into latitude and longitude
data[['Order Latitude', 'Order Longitude']] = pd.DataFrame(data['Coordinates'].tolist(), index=data.index)
data.drop(['Coordinates'], axis=1, inplace=True)
# Change name of Latitude and Longitude columns
data.rename(columns={'Latitude':'Customer Latitude', 'Longitude':'Customer Longitude'}, inplace=True)
data

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Customer Latitude,Customer Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,order_date,order_year,order_month,order_day,Order Latitude,Order Longitude
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.110000,0.04,180517,327.750000,0.29,1,327.750000,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,2/3/2018 22:56,Standard Class,2018-01-31 22:56:00,2018,1,31,-6.234986,106.994544
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.389999,0.05,179254,327.750000,-0.80,1,327.750000,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/18/2018 12:27,Standard Class,2018-01-13 12:27:00,2018,1,13,28.015929,73.317137
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.030001,0.06,179253,327.750000,-0.80,1,327.750000,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/17/2018 12:06,Standard Class,2018-01-13 12:06:00,2018,1,13,28.015929,73.317137
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.940001,0.07,179252,327.750000,0.08,1,327.750000,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/16/2018 11:45,Standard Class,2018-01-13 11:45:00,2018,1,13,-19.256939,146.823954
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.500000,0.09,179251,327.750000,0.45,1,327.750000,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/15/2018 11:24,Standard Class,2018-01-13 11:24:00,2018,1,13,-19.256939,146.823954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,0,45,Fishing,Brooklyn,EE. UU.,XXXXXXXXX,Maria,1005,Peterson,XXXXX

In [8]:
def calculate_distance(row):
    customer_coords = (row['Customer Latitude'], row['Customer Longitude'])
    order_coords = (row['Order Latitude'], row['Order Longitude'])
    return great_circle(customer_coords, order_coords).kilometers
# Calculate the distance between the customer and the order
data['Distance (km)'] = data.apply(calculate_distance, axis=1)
# Calculate the distance between the customer and the order
data['Distance (km)'] = data.apply(calculate_distance, axis=1)
data

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Customer Latitude,Customer Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,order_date,order_year,order_month,order_day,Order Latitude,Order Longitude,Distance (km)
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.110000,0.04,180517,327.750000,0.29,1,327.750000,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,2/3/2018 22:56,Standard Class,2018-01-31 22:56:00,2018,1,31,-6.234986,106.994544,18480.084962
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.389999,0.05,179254,327.750000,-0.80,1,327.750000,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/18/2018 12:27,Standard Class,2018-01-13 12:27:00,2018,1,13,28.015929,73.317137,13260.655956
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.030001,0.06,179253,327.750000,-0.80,1,327.750000,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/17/2018 12:06,Standard Class,2018-01-13 12:06:00,2018,1,13,28.015929,73.317137,12581.953241
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.940001,0.07,179252,327.750000,0.08,1,327.750000,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/16/2018 11:45,Standard Class,2018-01-13 11:45:00,2018,1,13,-19.256939,146.823954,11627.746132
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.500000,0.09,179251,327.750000,0.45,1,327.750000,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/15/2018 11:24,Standard Class,2018-01-13 11:24:00,2018,1,13,-19.256939,146.823954,16558.468052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011

Grab features

In [9]:
print(data.columns)
print(len(data.columns))
# Drop all Id's because not useful for model, drop the location columns because we have the coordinates, drop the date columns because we have the year, month, and day columns
# Also drop some profit columns because they are duplicates
unuseful_features = ["Delivery Status","Category Id","Customer City", "Customer Country","Customer Email","Customer Fname","Customer Id","Customer Lname","Customer Password",
                     "Customer State","Customer Street","Customer Zipcode","Department Id","Order City","Order Country","Order Customer Id","Order Item Cardprod Id","Order Item Discount Rate","Order Item Id",
                     "Order Item Profit Ratio","Order Profit Per Order", "Order Region","Order State","Order Zipcode","Product Card Id","Product Category Id","Product Description","Product Image", "Product Price",
                     "Product Status", "shipping date (DateOrders)","order date (DateOrders)", 'order_date']
print(len(unuseful_features))

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Customer Latitude',
       'Customer Longitude', 'Market', 'Order City', 'Order Country',
       'Order Customer Id', 'order date (DateOrders)', 'Order Id',
       'Order Item Cardprod Id', 'Order Item Discount',
       'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price',
       'Order Item Profit Ratio', 'Order Item Quantity', 'Sales',
       'Order Item Total', 'Order Profit Per Order', 'Order Region',
       'Order State', 'Order Status', 'Order Zipcode', 'Product Card Id',
       'Product Categor

In [10]:
useful_features = [col for col in data.columns if col not in unuseful_features]
print(len(useful_features))
print(useful_features)

27
['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'Late_delivery_risk', 'Category Name', 'Customer Segment', 'Department Name', 'Customer Latitude', 'Customer Longitude', 'Market', 'Order Id', 'Order Item Discount', 'Order Item Product Price', 'Order Item Quantity', 'Sales', 'Order Item Total', 'Order Status', 'Product Name', 'Shipping Mode', 'order_year', 'order_month', 'order_day', 'Order Latitude', 'Order Longitude', 'Distance (km)']


In [11]:
new_data = data[useful_features]
statuses_to_drop = ['CANCELED', 'SUSPECTED_FRAUD']
# Drop rows where late delivery risk is always 0 (when the order status is 'CANCELED' or 'SUSPECTED_FRAUD')
model_data = new_data[~new_data['Order Status'].isin(statuses_to_drop)]
# Drop rows where 'Order Status' is in the list of statuses to drop
model_data.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Late_delivery_risk,Category Name,Customer Segment,Department Name,Customer Latitude,Customer Longitude,Market,Order Id,Order Item Discount,Order Item Product Price,Order Item Quantity,Sales,Order Item Total,Order Status,Product Name,Shipping Mode,order_year,order_month,order_day,Order Latitude,Order Longitude,Distance (km)
0,DEBIT,3,4,91.250000,314.640015,0,Sporting Goods,Consumer,Fitness,18.251453,-66.037056,Pacific Asia,77202,13.110000,327.75,1,327.75,314.640015,COMPLETE,Smart watch,Standard Class,2018,1,31,-6.234986,106.994544,18480.084962
1,TRANSFER,5,4,-249.089996,311.359985,1,Sporting Goods,Consumer,Fitness,18.279451,-66.037064,Pacific Asia,75939,16.389999,327.75,1,327.75,311.359985,PENDING,Smart watch,Standard Class,2018,1,13,28.015929,73.317137,13260.655956
2,CASH,4,4,-247.779999,309.720001,0,Sporting Goods,Consumer,Fitness,37.292233,-121.881279,Pacific Asia,75938,18.030001,327.75,1,327.75,309.720001,CLOSED,Smart watch,Standard Class,2018,1,13,28.015929,73.317137,12581.953241
3,DEBIT,3,4,22.860001,304.809998,0,Sporting Goods,Home Office,Fitness,34.125946,-118.291016,Pacific Asia,75937,22.940001,327.75,1,327.75,304.809998,COMPLETE,Smart watch,Standard Class,2018,1,13,-19.256939,146.823954,11627.746132
4,PAYMENT,2,4,134.210007,298.250000,0,Sporting Goods,Corporate,Fitness,18.253769,-66.037048,Pacific Asia,75936,29.500000,327.75,1,327.75,298.250000,PENDING_PAYMENT,Smart watch,Standard Class,2018,1,13,-19.256939,146.823954,16558.468052


In [12]:
#One-Hot encoding categotical variables in the data
model_data = pd.get_dummies(model_data)
model_data.shape
model_data

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Late_delivery_risk,Customer Latitude,Customer Longitude,Order Id,Order Item Discount,Order Item Product Price,Order Item Quantity,Sales,Order Item Total,order_year,order_month,order_day,Order Latitude,Order Longitude,Distance (km),Type_CASH,Type_DEBIT,Type_PAYMENT,Type_TRANSFER,Category Name_Accessories,Category Name_As Seen on TV!,Category Name_Baby,Category Name_Baseball & Softball,Category Name_Basketball,Category Name_Books,Category Name_Boxing & MMA,Category Name_CDs,Category Name_Cameras,Category Name_Camping & Hiking,Category Name_Cardio Equipment,Category Name_Children's Clothing,Category Name_Cleats,Category Name_Computers,Category Name_Consumer Electronics,Category Name_Crafts,Category Name_DVDs,Category Name_Electronics,Category Name_Fishing,Category Name_Fitness Accessories,Category Name_Garden,Category Name_Girls' Apparel,Category Name_Golf Apparel,Category Name_Golf Bags & Carts,Category Name_Golf Balls,Category Name_Golf Gloves,Category Name_Golf Shoes,Category Name_Health and Beauty,Category Name_Hockey,Category Name_Hunting & Shooting,Category Name_Indoor/Outdoor Games,Category Name_Kids' Golf Clubs,Category Name_Lacrosse,Category Name_Men's Clothing,Category Name_Men's Footwear,Category Name_Men's Golf Clubs,Category Name_Music,Category Name_Pet Supplies,Category Name_Shop By Sport,Category Name_Soccer,Category Name_Sporting Goods,Category Name_Strength Training,Category Name_Tennis & Racquet,Category Name_Toys,Category Name_Trade-In,Category Name_Video Games,Category Name_Water Sports,Category Name_Women's Apparel,Category Name_Women's Clothing,Category Name_Women's Golf Clubs,Customer Segment_Consumer,Customer Segment_Corporate,Customer Segment_Home Office,Department Name_Apparel,Department Name_Book Shop,Department Name_Discs Shop,Department Name_Fan Shop,Department Name_Fitness,Department Name_Footwear,Department Name_Golf,Department Name_Health and Beauty,Department Name_Outdoors,Department Name_Pet Shop,Department Name_Technology,Market_Africa,Market_Europe,Market_LATAM,Market_Pacific Asia,Market_USCA,Order Status_CLOSED,Order Status_COMPLETE,Order Status_ON_HOLD,Order Status_PAYMENT_REVIEW,Order Status_PENDING,Order Status_PENDING_PAYMENT,Order Status_PROCESSING,Product Name_Adult dog supplies,Product Name_Baby sweater,Product Name_Bag Boy Beverage Holder,Product Name_Bag Boy M330 Push Cart,Product Name_Bowflex SelectTech 1090 Dumbbells,Product Name_Bridgestone e6 Straight Distance NFL Carolina,Product Name_Bridgestone e6 Straight Distance NFL San Dieg,Product Name_Bridgestone e6 Straight Distance NFL Tennesse,Product Name_Brooks Women's Ghost 6 Running Shoe,Product Name_Bushnell Pro X7 Jolt Slope Rangefinder,Product Name_CDs of rock,Product Name_Children's heaters,Product Name_Cleveland Golf Collegiate My Custom Wedge 588,Product Name_Cleveland Golf Women's 588 RTX CB Satin Chrom,Product Name_Clicgear 8.0 Shoe Brush,Product Name_Clicgear Rovic Cooler Bag,Product Name_Columbia Men's PFG Anchor Tough T-Shirt,Product Name_DVDs,Product Name_Dell Laptop,Product Name_Diamondback Boys' Insight 24 Performance Hybr,Product Name_Diamondback Girls' Clarity 24 Hybrid Bike 201,Product Name_Diamondback Women's Serene Classic Comfort Bi,Product Name_ENO Atlas Hammock Straps,Product Name_Elevation Training Mask 2.0,Product Name_Field & Stream Sportsman 16 Gun Fire Safe,Product Name_Fighting video games,Product Name_First aid kit,Product Name_Fitbit The One Wireless Activity & Sleep Trac,Product Name_Garmin Approach S3 Golf GPS Watch,Product Name_Garmin Approach S4 Golf GPS Watch,Product Name_Garmin Forerunner 910XT GPS Watch,Product Name_Glove It Imperial Golf Towel,Product Name_Glove It Urban Brick Golf Towel,Product Name_Glove It Women's Imperial Golf Glove,Product Name_Glove It Women's Mod Oval 3-Zip Carry All Gol,Product Name_Glove It Women's Mod Oval Golf Glove,Product Name_GoPro HERO3+ Black Edition Camera,Product Name_

In [13]:
# The columns we need to sum if we want to have one row per order
num_item_columns = ["Order Item Discount", "Order Item Product Price",
                        "Sales", 'Order Item Total', 'Benefit per order', 'Sales per customer']

cat_item_columns = ["Department Name", "Category Name"]

static_columns = [col for col in model_data.columns if col not in num_item_columns + cat_item_columns + ["Order Id"]]

In [14]:
product_columns = [col for col in model_data.columns if col.startswith('Product Name_')]
# Multiply product dummies with 'Order Item Quantity' to get the quantity per product
product_quantities = model_data[product_columns].multiply(model_data['Order Item Quantity'], axis=0)
# Rename the columns from 'Product Name_X' to 'Product Quantity_X'
product_quantities.columns = [col.replace('Product Name', 'Product Quantity') for col in product_columns]
# Sum the quantities across all items for each order
product_quantities_agg = product_quantities.groupby(model_data['Order Id']).sum()

# Step 2: Sum numeric columns
# Assuming you know which columns are numeric and specific to each order item
# Exclude 'Order Item Quantity' as it's already handled in the first step
numeric_agg = model_data.groupby('Order Id')[num_item_columns].sum()

# Step 3: Aggregate Categorical Columns
# Filter columns related to categorical dummies
categorical_columns = []
for col in cat_item_columns:
    categorical_columns.extend([col for col in model_data.columns if col.startswith(col+'_')])
# Aggregate categorical columns with a max, so that any 1 in the order is retained
categorical_agg = model_data.groupby('Order Id')[categorical_columns].max()

# Combine all aggregated dataframes
agg_df = pd.concat([product_quantities_agg, numeric_agg, categorical_agg], axis=1)

# Now, keep columns that are the same for each order and join with aggregated data
# Assuming the columns 'order_date', 'customer_id', etc., are the same across order items
static_data = model_data.groupby('Order Id').first()[static_columns]

# Combine the static data with the aggregated data
prep_data = pd.concat([static_data, agg_df], axis=1)
# Delete 'Order Item Quantity', and columns starting with 'Product Name' and cat_item_columns
prep_data.drop(columns=['Order Item Quantity'] + product_columns + categorical_columns, inplace=True)
# Change the name of num_item_columns to be more descriptive
prep_data.rename(columns={'Order Item Discount': 'Total Order Item Discount',
                            'Order Item Product Price': 'Total Order Item Product Price',
                            'Sales': 'Total Sales', 'Order Item Total': 'Total Order Item Total',
                            'Benefit per order': 'Total Benefit per Order',
                            'Sales per customer': 'Total Sales per Customer'}, inplace=True)
prep_data

,Days for shipping (real),Days for shipment (scheduled),Late_delivery_risk,Customer Latitude,Customer Longitude,order_year,order_month,order_day,Order Latitude,Order Longitude,Distance (km),Type_CASH,Type_DEBIT,Type_PAYMENT,Type_TRANSFER,Category Name_Accessories,Category Name_As Seen on TV!,Category Name_Baby,Category Name_Baseball & Softball,Category Name_Basketball,Category Name_Books,Category Name_Boxing & MMA,Category Name_CDs,Category Name_Cameras,Category Name_Camping & Hiking,Category Name_Cardio Equipment,Category Name_Children's Clothing,Category Name_Cleats,Category Name_Computers,Category Name_Consumer Electronics,Category Name_Crafts,Category Name_DVDs,Category Name_Electronics,Category Name_Fishing,Category Name_Fitness Accessories,Category Name_Garden,Category Name_Girls' Apparel,Category Name_Golf Apparel,Category Name_Golf Bags & Carts,Category Name_Golf Balls,Category Name_Golf Gloves,Category Name_Golf Shoes,Category Name_Health and Beauty,Category Name_Hockey,Category Name_Hunting & Shooting,Category Name_Indoor/Outdoor Games,Category Name_Kids' Golf Clubs,Category Name_Lacrosse,Category Name_Men's Clothing,Category Name_Men's Footwear,Category Name_Men's Golf Clubs,Category Name_Music,Category Name_Pet Supplies,Category Name_Shop By Sport,Category Name_Soccer,Category Name_Sporting Goods,Category Name_Strength Training,Category Name_Tennis & Racquet,Category Name_Toys,Category Name_Trade-In,Category Name_Video Games,Category Name_Water Sports,Category Name_Women's Apparel,Category Name_Women's Clothing,Category Name_Women's Golf Clubs,Customer Segment_Consumer,Customer Segment_Corporate,Customer Segment_Home Office,Department Name_Apparel,Department Name_Book Shop,Department Name_Discs Shop,Department Name_Fan Shop,Department Name_Fitness,Department Name_Footwear,Department Name_Golf,Department Name_Health and Beauty,Department Name_Outdoors,Department Name_Pet Shop,Department Name_Technology,Market_Africa,Market_Europe,Market_LATAM,Market_Pacific Asia,Market_USCA,Order Status_CLOSED,Order Status_COMPLETE,Order Status_ON_HOLD,Order Status_PAYMENT_REVIEW,Order Status_PENDING,Order Status_PENDING_PAYMENT,Order Status_PROCESSING,Shipping Mode_First Class,Shipping Mode_Same Day,Shipping Mode_Second Class,Shipping Mode_Standard Class,Product Quantity_Adult dog supplies,Product Quantity_Baby sweater,Product Quantity_Bag Boy Beverage Holder,Product Quantity_Bag Boy M330 Push Cart,Product Quantity_Bowflex SelectTech 1090 Dumbbells,Product Quantity_Bridgestone e6 Straight Distance NFL Carolina,Product Quantity_Bridgestone e6 Straight Distance NFL San Dieg,Product Quantity_Bridgestone e6 Straight Distance NFL Tennesse,Product Quantity_Brooks Women's Ghost 6 Running Shoe,Product Quantity_Bushnell Pro X7 Jolt Slope Rangefinder,Product Quantity_CDs of rock,Product Quantity_Children's heaters,Product Quantity_Cleveland Golf Collegiate My Custom Wedge 588,Product Quantity_Cleveland Golf Women's 588 RTX CB Satin Chrom,Product Quantity_Clicgear 8.0 Shoe Brush,Product Quantity_Clicgear Rovic Cooler Bag,Product Quantity_Columbia Men's PFG Anchor Tough T-Shirt,Product Quantity_DVDs,Product Quantity_Dell Laptop,Product Quantity_Diamondback Boys' Insight 24 Performance Hybr,Product Quantity_Diamondback Girls' Clarity 24 Hybrid Bike 201,Product Quantity_Diamondback Women's Serene Classic Comfort Bi,Product Quantity_ENO Atlas Hammock Straps,Product Quantity_Elevation Training Mask 2.0,Product Quantity_Field & Stream Sportsman 16 Gun Fire Safe,Product Quantity_Fighting video games,Product Quantity_First aid kit,Product Quantity_Fitbit The One Wireless Activity & Sleep Trac,Product Quantity_Garmin Approach S3 Golf GPS Watch,Product Quantity_Garmin Approach S4 Golf GPS Watch,Product Quantity_Garmin Forerunner 910XT GPS Watch,Product Quantity_Glove It Imperial Golf Towel,Product Quantity_Glove It Urban Brick Golf Towel,Product Quantity_Glove It Women's Imperial Golf Glove,Product Quantity_Glove It Women's Mod Oval 3-Zip Carry All Gol

# 4) Save data

In [15]:
# Save the data
prep_data.to_csv('Data/prep_data.csv', index=False)